In [82]:
import pandas as pd
import numpy as np
import dateutil.easter
from prophet import Prophet
from itertools import product
from prophet.diagnostics import cross_validation, performance_metrics
from IPython.display import display
from typing import Optional, List, Dict, Tuple

In [83]:
df_national = pd.read_csv(
    '../data/processed/data_processed_v3_National_NoAcum_Total.csv')
df_national.head()

,Year,Week,Date,Epi_Year,total_cases
0,2014,2,2014-01-13,2014,4.0
1,2014,3,2014-01-20,2014,29.0
2,2014,4,2014-01-27,2014,47.0
3,2014,5,2014-02-03,2014,36.0
4,2014,6,2014-02-10,2014,42.0


In [84]:
# Prepare dataframe for Prophet
df_national = df_national.rename(columns={'Date': 'ds', 'total_cases': 'y'})
df_national['ds'] = pd.to_datetime(df_national['ds'])
df_national = df_national.sort_values('ds')
print("DataFrame prepared for Prophet - Head:")
print(df_national.head())
print("")
print("DataFrame prepared for Prophet - Tail:")
print(df_national.tail())

DataFrame prepared for Prophet - Head:
   Year  Week         ds  Epi_Year     y
0  2014     2 2014-01-13      2014   4.0
1  2014     3 2014-01-20      2014  29.0
2  2014     4 2014-01-27      2014  47.0
3  2014     5 2014-02-03      2014  36.0
4  2014     6 2014-02-10      2014  42.0

DataFrame prepared for Prophet - Tail:
     Year  Week         ds  Epi_Year     y
568  2024    48 2024-11-25      2024  33.0
569  2024    49 2024-12-02      2024  49.0
570  2024    50 2024-12-09      2024  38.0
571  2024    51 2024-12-16      2024  32.0
572  2024    52 2024-12-23      2024  36.0


In [85]:
df_states_original = pd.read_csv('../data/processed/data_processed_v3_NoAcum_Total.csv')
df_states_sex_original = pd.read_csv('../data/processed/data_processed_v3_NoAcum_MF.csv')
states_list = df_states_original['Entity'].unique().tolist()
try:
    states_list.remove('National')
except ValueError:
    pass

In [86]:
# Prepare dataframes for each state
dfs_states = {}
for state in states_list:
    df_state = df_states_original[df_states_original['Entity'] == state].copy()
    df_state = df_state.groupby('Date')['total_cases'].sum().reset_index()
    df_state = df_state.rename(columns={'Date': 'ds', 'total_cases': 'y'})
    df_state['ds'] = pd.to_datetime(df_state['ds'])
    df_state = df_state.sort_values('ds')
    dfs_states[state] = df_state

print("Prepared DataFrames for each state:")
for i, (state, df) in enumerate(dfs_states.items()):
    print(f"State: {state}")
    print(df.head())
    print("")
    if i == 5: 
        break

Prepared DataFrames for each state:
State: Aguascalientes
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  0.0
3 2014-02-03  0.0
4 2014-02-10  0.0

State: Baja California
          ds    y
0 2014-01-13  0.0
1 2014-01-20  6.0
2 2014-01-27  1.0
3 2014-02-03  3.0
4 2014-02-10  4.0

State: Baja California Sur
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  0.0
3 2014-02-03  0.0
4 2014-02-10  0.0

State: Campeche
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  1.0
3 2014-02-03  1.0
4 2014-02-10  3.0

State: Chiapas
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  1.0
3 2014-02-03  1.0
4 2014-02-10  2.0

State: Chihuahua
          ds    y
0 2014-01-13  0.0
1 2014-01-20  2.0
2 2014-01-27  2.0
3 2014-02-03  2.0
4 2014-02-10  0.0



In [87]:
# Prepare dataframes for each state and sex
dfs_states_sex = {}
for state in states_list:
    for sex in ['M', 'F']: # 'M' = Male, 'F' = Female
        df_ss = df_states_sex_original[df_states_sex_original['Entity'] == state].copy()
        df_ss = df_ss.groupby('Date')[sex].sum().reset_index()
        df_ss = df_ss.rename(columns={'Date': 'ds', sex: 'y'})
        df_ss['ds'] = pd.to_datetime(df_ss['ds'])
        df_ss = df_ss.sort_values('ds')
        dfs_states_sex[(state, sex)] = df_ss
   

print("Prepared DataFrames for each state:")
for i, (state, df) in enumerate(dfs_states_sex.items()):
    print(f"State: {state}")
    print(df.head())
    print("")
    if i == 5:
        break

Prepared DataFrames for each state:
State: ('Aguascalientes', 'M')
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  0.0
3 2014-02-03  0.0
4 2014-02-10  0.0

State: ('Aguascalientes', 'F')
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  0.0
3 2014-02-03  0.0
4 2014-02-10  0.0

State: ('Baja California', 'M')
          ds    y
0 2014-01-13  0.0
1 2014-01-20  1.0
2 2014-01-27  0.0
3 2014-02-03  0.0
4 2014-02-10  1.0

State: ('Baja California', 'F')
          ds    y
0 2014-01-13  0.0
1 2014-01-20  5.0
2 2014-01-27  1.0
3 2014-02-03  3.0
4 2014-02-10  3.0

State: ('Baja California Sur', 'M')
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  0.0
3 2014-02-03  0.0
4 2014-02-10  0.0

State: ('Baja California Sur', 'F')
          ds    y
0 2014-01-13  0.0
1 2014-01-20  0.0
2 2014-01-27  0.0
3 2014-02-03  0.0
4 2014-02-10  0.0



In [88]:
# Holidays definition
years = range(2014, 2030) 
holidays_list = []
for year in years:
    holidays_list.append({'ds': f'{year}-12-25', 'holiday': 'Navidad'})
    holidays_list.append({'ds': f'{year}-01-01', 'holiday': 'Ano_Nuevo'})
    easter_date = dateutil.easter.easter(year)
    holidays_list.append({'ds': easter_date.strftime('%Y-%m-%d'), 'holiday': 'Semana_Santa'})
    friday_before_easter = easter_date - pd.Timedelta(days=2)
    holidays_list.append({'ds': friday_before_easter.strftime('%Y-%m-%d'), 'holiday': 'Viernes_Santo'})

# create dataframe once and set Semana_Santa window
holidays = pd.DataFrame(holidays_list)
holidays.loc[holidays['holiday'] == 'Semana_Santa', 'lower_window'] = -3
holidays.loc[holidays['holiday'] == 'Semana_Santa', 'upper_window'] = 3

In [89]:
def build_mex_holidays(start_year, end_year, add_windows=True):
    rows = []
    for y in range(start_year, end_year + 1):
        rows.append({"ds": pd.Timestamp(f"{y}-12-25"), "holiday": "Navidad"})
        rows.append({"ds": pd.Timestamp(f"{y}-01-01"), "holiday": "Ano_Nuevo"})
        easter= pd.Timestamp(dateutil.easter.easter(y))
        rows.append({"ds": easter, "holiday": "Semana_Santa"})
    hol = pd.DataFrame(rows)
    if add_windows:
        mask = hol['holiday'] == 'Semana_Santa'
        hol.loc[mask, 'lower_window'] = -3
        hol.loc[mask, 'upper_window'] = 3
    return hol


In [90]:
def add_covid_dummy(df, start_covid='2020-03-01', end_covid='2022-12-31 '):
    s = pd.Timestamp(start_covid)
    e = pd.Timestamp(end_covid)
    out = df.copy()
    out['covid_dummy'] = ((out['ds'] >= s) & (out['ds'] <= e)).astype(int)
    return out

In [91]:
# Covid-19 Indicator
start_covid = pd.Timestamp('2020-03-01')
end_covid = pd.Timestamp('2022-12-31') 
def covid_indicator(date):
    if start_covid <= date <= end_covid:
        return 1
    else:
        return 0


df_national['covid_dummy'] = df_national['ds'].apply(covid_indicator)
for state, df_state in dfs_states.items():
    df_state['covid_dummy'] = df_state['ds'].apply(covid_indicator)
for key, df_ss in dfs_states_sex.items():
    df_ss['covid_dummy'] = df_ss['ds'].apply(covid_indicator)

print("DataFrames with Covid-19 indicator added:")
print("National DataFrame - Head:")
print(df_national.head())
print("")
print("State DataFrame Example - Head:")
example_state = list(dfs_states.keys())[0]
print(f"State: {example_state}")
print(dfs_states[example_state].head())
print("")
print("State and Sex DataFrame Example - Head:")
example_key = list(dfs_states_sex.keys())[0]
print(f"State and Sex   : {example_key}")
print(dfs_states_sex[example_key].head())


DataFrames with Covid-19 indicator added:
National DataFrame - Head:
   Year  Week         ds  Epi_Year     y  covid_dummy
0  2014     2 2014-01-13      2014   4.0            0
1  2014     3 2014-01-20      2014  29.0            0
2  2014     4 2014-01-27      2014  47.0            0
3  2014     5 2014-02-03      2014  36.0            0
4  2014     6 2014-02-10      2014  42.0            0

State DataFrame Example - Head:
State: Aguascalientes
          ds    y  covid_dummy
0 2014-01-13  0.0            0
1 2014-01-20  0.0            0
2 2014-01-27  0.0            0
3 2014-02-03  0.0            0
4 2014-02-10  0.0            0

State and Sex DataFrame Example - Head:
State and Sex   : ('Aguascalientes', 'M')
          ds    y  covid_dummy
0 2014-01-13  0.0            0
1 2014-01-20  0.0            0
2 2014-01-27  0.0            0
3 2014-02-03  0.0            0
4 2014-02-10  0.0            0


In [92]:
# Helper to build a Prophet model with our common settings and a parameter dict
def _build_prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False, seasonality_mode='multiplicative', changepoint_prior_scale=0.1, seasonality_prior_scale=5.0, holidays_prior_scale=0.5, changepoint_range=0.8, holidays: Optional[pd.DataFrame] = None, add_regressors: Optional[List[str]] = None):
    m = Prophet(yearly_seasonality=yearly_seasonality, weekly_seasonality=weekly_seasonality, daily_seasonality=daily_seasonality, seasonality_mode=seasonality_mode,
                changepoint_prior_scale=changepoint_prior_scale, seasonality_prior_scale=seasonality_prior_scale, holidays_prior_scale=holidays_prior_scale, changepoint_range=changepoint_range, holidays=holidays)
    if add_regressors:
        for reg in add_regressors:
            m.add_regressor(reg)
    return m

In [93]:
def _prophet_cv_smape(model, df, horizon_weeks=52, initial_years=3, period_weeks=26):
    # horizon_weeks: number of weeks to forecast in each fold
    # initial_years: number of years for the initial training period
    # period_weeks: spacing between cutoff dates
    
    horizon = f"{7*horizon_weeks} days"
    initial = f"{365*initial_years} days"
    period = f"{7*period_weeks} days"

    cv_df = cross_validation(model, horizon=horizon, initial=initial, period=period, parallel="processes")
    perf = performance_metrics(cv_df, rolling_window=1)

    return perf['smape'].mean()

In [94]:
def tune_prophet_for_df(df, holidays, add_reggresors: Optional[List[str]] = None, param_grid: Optional[Dict[str, List]] = None, random_k: Optional[int] = None, horizon_weeks=52, intial_years=3, period_weeks=26, verbose=True):
    if param_grid is None:
        param_grid = {
            "seasonality_mode": ["multiplicative", "additive"],
            "changepoint_prior_scale": [0.01, 0.1, 0.5],
            "seasonality_prior_scale": [1.0, 5.0, 10.0],
            "holidays_prior_scale": [1.0, 5.0, 10.0],
            "changepoint_range": [0.7, 0.8, 0.9],
        }
    keys, values = zip(*param_grid.items())
    all_param_combinations = [dict(zip(keys, v)) for v in product(*values)]
    if random_k is not None and random_k < len(all_param_combinations):
        np.random.shuffle(all_param_combinations)
        combos = all_param_combinations[:random_k]
    else:
        combos = all_param_combinations
    
    best_params, best_model, best_smape = None, None, float('inf')

    for i, p in enumerate(combos, start=1):
        try:
            if verbose:
                print(f"[tuning] combination {i}/{len(combos)} -> {p}")
            model = _build_prophet(
                yearly_seasonality=True,
                weekly_seasonality=False,
                daily_seasonality=False,
                seasonality_mode=p['seasonality_mode'],
                changepoint_prior_scale=p['changepoint_prior_scale'],
                seasonality_prior_scale=p['seasonality_prior_scale'],
                holidays_prior_scale=p['holidays_prior_scale'],
                changepoint_range=p['changepoint_range'],
                holidays=holidays,
                add_regressors=add_reggresors
            )
            model.fit(df)

            smape = _prophet_cv_smape(
                model, df, horizon_weeks=horizon_weeks, initial_years=intial_years, period_weeks=period_weeks
            )

            if verbose:
                print(f"    -> SMAPE: {smape:.4f}")
            if smape < best_smape:
                best_smape = smape
                best_params = p
                best_model = model
        except Exception as e:
            if verbose:
                print(f"    !!! Exception during tuning: {e}")
            continue

    
    if best_model is None:
        raise RuntimeError("No valid model found during tuning.")
    
    if verbose:
        print("\nBest hyperparameters found:", best_params)
        print(f"With SMAPE: {best_smape:.4f}")
    return best_params, best_model, best_smape

In [95]:
def fit_and_forecast(df,params,holidays,add_regressors: Optional[List[str]],horizon_weeks=260,freq="W-MON"):
    m = _build_prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False,
        seasonality_mode=params.get("seasonality_mode", "multiplicative"),
        changepoint_prior_scale=params.get("changepoint_prior_scale", 0.1),
        seasonality_prior_scale=params.get("seasonality_prior_scale", 5.0),
        holidays_prior_scale=params.get("holidays_prior_scale", 5.0),
        changepoint_range=params.get("changepoint_range", 0.8),
        holidays=holidays,
        add_regressors=add_regressors
    )
    m.fit(df)
    future = m.make_future_dataframe(periods=horizon_weeks, freq=freq)
    # Extend regressors into the future
    for reg in (add_regressors or []):
        if reg == "covid_dummy":
            # por diseño la pandemia no continúa → 0 en el futuro
            tmp = add_covid_dummy(future[["ds"]])
            future["covid_dummy"] = tmp["covid_dummy"].values
        else:
            raise ValueError(f"Extend regressor into the future for: {reg}")
    forecast = m.predict(future)
    return m, forecast

In [96]:
def hierarchical_tuning_and_forecast(df_national, dfs_states, dfs_states_sex,start_year = None,end_year = None, covid_start = "2020-03-01",covid_end = "2022-12-31",horizon_weeks_cv = 52, initial_years_cv = 3, period_weeks_cv = 26, horizon_weeks_forecast = 260, grid_national : Optional[Dict[str, List]] = None, random_k_state: int=8, micro_tune_sex = False, verbose = True,):
    # Holidays for the whole range
    if start_year is None:
        start_year = pd.to_datetime(df_national["ds"]).min().year
    if end_year is None:
        end_year = pd.to_datetime(df_national["ds"]).min().year + 5
    
    holidays = build_mex_holidays(start_year, end_year, add_windows=True)

    df_national = add_covid_dummy(df_national, covid_start, covid_end)
    dfs_states = {s: add_covid_dummy(df, covid_start, covid_end) for s, df in dfs_states.items()}
    dfs_states_sex = {(s, sex) : add_covid_dummy(df, covid_start, covid_end) for (s, sex), df in dfs_states_sex.items()}

    add_regs = ["covid_dummy"]

    # National Level

    if verbose:
        print("National Level Tunning")
    best_params_nat, best_model_nat, best_smape_nat = tune_prophet_for_df(df_national, holidays, add_regs, grid_national, None, horizon_weeks_cv, initial_years_cv, period_weeks_cv, verbose)
    if verbose:
        print("National BEST:", best_params_nat, "sMAPE:", round(best_smape_nat, 3))
    if verbose:
        print("National Level Forecast")
    model_nat, forecast_nat = fit_and_forecast(df_national, best_params_nat, holidays, add_regs, horizon_weeks_forecast)

    # State Level
    if verbose:
        print("State Level Tunning")
    params_states = {}
    forecasts_states = {}
    for state, df_s in dfs_states.items():
        p_state, _, sm = tune_prophet_for_df(df_s, holidays, add_regs, None, random_k_state, horizon_weeks_cv, initial_years_cv, period_weeks_cv, False)
        params_states[state] = p_state
        if verbose:
            print(f"[{state}] sMAPE={sm:.2f} params={p_state}")
        
        # Forecast with choose params
        _, f_s = fit_and_forecast(df_s, params_states[state], holidays, add_regs, horizon_weeks_forecast)
        forecasts_states[state] = f_s
    
    # State - Sex Level
    if verbose: 
        print("State - Sex Level Tunning")
    params_states_sex: Dict[Tuple[str,str], Dict] = {}
    forecasts_states_sex: Dict[Tuple[str,str], pd.DataFrame] = {}

    for (state, sex), df_ss in dfs_states_sex.items():
        base_params = params_states.get(state, best_params_nat).copy()
        chosen_params = base_params

        # Micro-tuning 
        if micro_tune_sex and len(df_ss) >= 52*(initial_years_cv+1):
            tiny_grid = {
                "seasonality_mode": [base_params.get("seasonality_mode", "multiplicative")],
                "changepoint_prior_scale": [base_params["changepoint_prior_scale"]*f
                                            for f in [0.5, 1.0, 2.0]],
                "seasonality_prior_scale": [base_params["seasonality_prior_scale"]],
                "holidays_prior_scale": [base_params["holidays_prior_scale"]],
                "changepoint_range": [base_params.get("changepoint_range", 0.8)]
            }
            try:
                p_ss, _, sm_ss = tune_prophet_for_df(df_ss, holidays, add_regs, tiny_grid, None, horizon_weeks_cv, initial_years_cv, period_weeks_cv,False)
                chosen_params = p_ss
                if verbose:
                    print(f"[{state}-{sex}] micro-tune sMAPE≈{sm_ss:.2f} params={p_ss}")
            except Exception as e:
                if verbose:
                    print(f"[{state}-{sex}] micro-tune failed ({e}). Use params from state")

        params_states_sex[(state, sex)] = chosen_params
        _, f_ss = fit_and_forecast(df_ss, chosen_params, holidays,add_regs, horizon_weeks_forecast)
        forecasts_states_sex[(state, sex)] = f_ss

    return {
        "national": {
            "params": best_params_nat, "forecast": forecast_nat
        },
        "states": {
            "params": params_states, "forecasts": forecasts_states
        },
        "states_sex": {
            "params": params_states_sex, "forecasts": forecasts_states_sex
        },
        "holidays": holidays
    }

    

In [97]:
national_grid_opt = {
    "seasonality_mode": ["multiplicative", "additive"],
    "changepoint_prior_scale": [0.05, 0.1, 0.2],
    "seasonality_prior_scale": [1.0, 5.0, 10.0],
    "holidays_prior_scale": [1.0, 5.0],
    "changepoint_range": [0.8, 0.95],
}

res = hierarchical_tures = hierarchical_tuning_and_forecast(
    df_national=df_national,
    dfs_states=dfs_states,
    dfs_states_sex=dfs_states_sex,
    grid_national=national_grid_opt,
    random_k_state=8,      
    micro_tune_sex=False,   
    horizon_weeks_cv=52,
    initial_years_cv=3,
    period_weeks_cv=26,
    horizon_weeks_forecast=260,
    verbose=True
)

forecast_nat = res["national"]["forecast"]         
forecasts_states = res["states"]["forecasts"]         
forecasts_states_sex = res["states_sex"]["forecasts"] 

National Level Tunning
[tuning] combination 1/72 -> {'seasonality_mode': 'multiplicative', 'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 1.0, 'holidays_prior_scale': 1.0, 'changepoint_range': 0.8}


12:48:07 - cmdstanpy - INFO - Chain [1] start processing
12:48:07 - cmdstanpy - INFO - Chain [1] done processing
12:48:09 - cmdstanpy - INFO - Chain [1] start processing
12:48:10 - cmdstanpy - INFO - Chain [1] start processing
12:48:10 - cmdstanpy - INFO - Chain [1] start processing
12:48:10 - cmdstanpy - INFO - Chain [1] start processing
12:48:10 - cmdstanpy - INFO - Chain [1] start processing
12:48:10 - cmdstanpy - INFO - Chain [1] start processing
12:48:10 - cmdstanpy - INFO - Chain [1] start processing
12:48:10 - cmdstanpy - INFO - Chain [1] done processing
12:48:10 - cmdstanpy - INFO - Chain [1] done processing
12:48:10 - cmdstanpy - INFO - Chain [1] done processing
12:48:10 - cmdstanpy - INFO - Chain [1] done processing
12:48:10 - cmdstanpy - INFO - Chain [1] done processing
12:48:10 - cmdstanpy - INFO - Chain [1] done processing
12:48:10 - cmdstanpy - INFO - Chain [1] done processing
12:48:10 - cmdstanpy - INFO - Chain [1] start processing
12:48:10 - cmdstanpy - INFO - Chain [1]

    -> SMAPE: 0.4066
[tuning] combination 2/72 -> {'seasonality_mode': 'multiplicative', 'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 1.0, 'holidays_prior_scale': 1.0, 'changepoint_range': 0.95}


12:48:10 - cmdstanpy - INFO - Chain [1] start processing
12:48:11 - cmdstanpy - INFO - Chain [1] done processing
12:48:12 - cmdstanpy - INFO - Chain [1] start processing
12:48:12 - cmdstanpy - INFO - Chain [1] done processing
12:48:12 - cmdstanpy - INFO - Chain [1] start processing
12:48:12 - cmdstanpy - INFO - Chain [1] start processing
12:48:12 - cmdstanpy - INFO - Chain [1] start processing
12:48:12 - cmdstanpy - INFO - Chain [1] start processing
12:48:12 - cmdstanpy - INFO - Chain [1] done processing
12:48:12 - cmdstanpy - INFO - Chain [1] start processing
12:48:12 - cmdstanpy - INFO - Chain [1] done processing
12:48:12 - cmdstanpy - INFO - Chain [1] start processing
12:48:12 - cmdstanpy - INFO - Chain [1] start processing
12:48:12 - cmdstanpy - INFO - Chain [1] done processing
12:48:12 - cmdstanpy - INFO - Chain [1] done processing
12:48:13 - cmdstanpy - INFO - Chain [1] done processing
12:48:13 - cmdstanpy - INFO - Chain [1] done processing
12:48:13 - cmdstanpy - INFO - Chain [1]

    -> SMAPE: 0.4123
[tuning] combination 3/72 -> {'seasonality_mode': 'multiplicative', 'changepoint_prior_scale': 0.05, 'seasonality_prior_scale': 1.0, 'holidays_prior_scale': 5.0, 'changepoint_range': 0.8}


...

13:01:55 - cmdstanpy - INFO - Chain [1] done processing
13:01:55 - cmdstanpy - INFO - Chain [1] start processing
13:01:55 - cmdstanpy - INFO - Chain [1] done processing
13:01:57 - cmdstanpy - INFO - Chain [1] start processing
13:01:57 - cmdstanpy - INFO - Chain [1] done processing
13:01:57 - cmdstanpy - INFO - Chain [1] start processing
13:01:57 - cmdstanpy - INFO - Chain [1] start processing
13:01:57 - cmdstanpy - INFO - Chain [1] done processing
13:01:57 - cmdstanpy - INFO - Chain [1] start processing
13:01:57 - cmdstanpy - INFO - Chain [1] done processing
13:01:57 - cmdstanpy - INFO - Chain [1] done processing
13:01:57 - cmdstanpy - INFO - Chain [1] start processing
13:01:57 - cmdstanpy - INFO - Chain [1] start processing
13:01:57 - cmdstanpy - INFO - Chain [1] done processing
13:01:57 - cmdstanpy - INFO - Chain [1] start processing
13:01:57 - cmdstanpy - INFO - Chain [1] done processing
13:01:57 - cmdstanpy - INFO - Chain [1] start processing
13:01:57 - cmdstanpy - INFO - Chain [1]

[Zacatecas] sMAPE=1.76 params={'seasonality_mode': 'multiplicative', 'changepoint_prior_scale': 0.01, 'seasonality_prior_scale': 10.0, 'holidays_prior_scale': 1.0, 'changepoint_range': 0.9}


13:02:19 - cmdstanpy - INFO - Chain [1] done processing
13:02:20 - cmdstanpy - INFO - Chain [1] start processing
13:02:20 - cmdstanpy - INFO - Chain [1] done processing


State - Sex Level Tunning


13:02:21 - cmdstanpy - INFO - Chain [1] start processing
13:02:21 - cmdstanpy - INFO - Chain [1] done processing
13:02:21 - cmdstanpy - INFO - Chain [1] start processing
13:02:21 - cmdstanpy - INFO - Chain [1] done processing
13:02:21 - cmdstanpy - INFO - Chain [1] start processing
13:02:21 - cmdstanpy - INFO - Chain [1] done processing
13:02:22 - cmdstanpy - INFO - Chain [1] start processing
13:02:22 - cmdstanpy - INFO - Chain [1] done processing
13:02:22 - cmdstanpy - INFO - Chain [1] start processing
13:02:22 - cmdstanpy - INFO - Chain [1] done processing
13:02:22 - cmdstanpy - INFO - Chain [1] start processing
13:02:22 - cmdstanpy - INFO - Chain [1] done processing
13:02:22 - cmdstanpy - INFO - Chain [1] start processing
13:02:23 - cmdstanpy - INFO - Chain [1] done processing
13:02:23 - cmdstanpy - INFO - Chain [1] start processing
13:02:23 - cmdstanpy - INFO - Chain [1] done processing
13:02:23 - cmdstanpy - INFO - Chain [1] start processing
13:02:23 - cmdstanpy - INFO - Chain [1]

In [98]:
# Structure forecasts into a better format that will be used later for reconciliation 
H = 260 # Forecast horizon in weeks
y_base = []

yhat_national = forecast_nat['yhat'].iloc[-260:].values
yhat_states = {}
for state in states_list:
    yhat_states[state] = forecasts_states[state]['yhat'].iloc[-260:].values

yhat_states_sex = {}
for state in states_list:
    for sex in ['M', 'F']:
        yhat_states_sex[(state, sex)] = forecasts_states_sex[(state, sex)]['yhat'].iloc[-260:].values

# National level
y_base.append(yhat_national)
# State level
for state in states_list:
    y_base.append(yhat_states[state])
# State and sex level
for state in states_list:
    y_base.append(yhat_states_sex[(state, 'M')])
    y_base.append(yhat_states_sex[(state, 'F')])

Y_base = np.vstack(y_base)  
print(Y_base.shape)  # Shape: (97, 260) -> 1 National + 32 States + 32 States * 2 Genders

(97, 260)


In [99]:
# Reconciliation with MinT
n = Y_base.shape[0]  # Total number of series
m = 64  # Number of series bottom
# Construct the summing matrix S
S = np.zeros((n, m))
# National level
S[0, :] = 1

# State level
for i, state in enumerate(states_list):
    col_M = 2*(i-1) # Males of state i 
    col_F = 2*(i-1) + 1 # Females of state i
    S[i, col_M] = 1
    S[i, col_F] = 1

# State and sex level
for j in range(m):
    S[1 + len(states_list)+j, j] = 1 # Bottom level series

print("Summing matrix S shape:", S.shape)


Summing matrix S shape: (97, 64)


In [100]:
# Proyection matrix for OLS reconciliation
inv_Sst = np.linalg.inv(S.T @ S) 
G_ols = inv_Sst @ S.T
Y_recon_bottom = G_ols @ Y_base
Y_reconciled = S @ Y_recon_bottom # Forecast Matrix (97 x H) 

# Now we can verify that the reconciliation worked
# Check National level
national_reconciliated_week1 = Y_reconciled[0, 0]
sum_states_week1 = np.sum(Y_reconciled[1:1+len(states_list), 0])
print(f"National reconciliated week 1: {national_reconciliated_week1}")
print(f"Sum states week 1: {sum_states_week1}")

print("")
# Check State level
for i, state in enumerate(states_list):
    state_reconciliated_week1 = Y_reconciled[1 + i, 0]
    col_M = 2*i
    col_F = 2*i + 1
    sum_state_reconciliated_week1 = Y_reconciled[1 + len(states_list) + col_M, 0] + Y_reconciled[1 + len(states_list) + col_F, 0]
    print(f"--- State: {state}")
    print(f"\tState reconciliated week 1: {state_reconciliated_week1}")
    print(f"\tSum state reconciliated week 1: {sum_state_reconciliated_week1}")
    if i == 5:  # Just check first 6 states
        break

National reconciliated week 1: 25.50646246013229
Sum states week 1: 26.65227750259543

--- State: Aguascalientes
	State reconciliated week 1: -0.025302170093994145
	Sum state reconciliated week 1: -0.025302170093994145
--- State: Baja California
	State reconciliated week 1: 1.5024859176689311
	Sum state reconciliated week 1: 1.5024859176689311
--- State: Baja California Sur
	State reconciliated week 1: -0.26893372368742496
	Sum state reconciliated week 1: -0.26893372368742496
--- State: Campeche
	State reconciliated week 1: -0.5876960934117307
	Sum state reconciliated week 1: -0.5876960934117307
--- State: Chiapas
	State reconciliated week 1: 0.20740015507464038
	Sum state reconciliated week 1: 0.20740015507464038
--- State: Chihuahua
	State reconciliated week 1: 2.122497842008566
	Sum state reconciliated week 1: 2.122497842008566
